## Import library

In [1]:
import mysql.connector
import pandas as pd

### Create Connection and Function

In [2]:
mydb=mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='********',
    database='world')

mycursor=mydb.cursor()
# crate acess to database
query='show tables;'
#query
mycursor.execute(query)
#execute query
result=mycursor.fetchall()
#save result
df=pd.DataFrame(result,columns=mycursor.column_names)
df.head()

,Tables_in_world
0,city
1,country
2,countrylanguage


In [3]:
mycursor=mydb.cursor()
def sql(query):
    mycursor.execute(query)
    result=mycursor.fetchall()
    df=pd.DataFrame(result,columns=mycursor.column_names)
    return df

## Ada berapa negara di data world?

,jumlah_negara
0,239


In [6]:
sql(''' 
SELECT DISTINCT COUNT(Name) as Jumlah_Negara FROM Country;
''')

,Jumlah_Negara
0,239


## Tampilkan rata-rata harapan hidup berdasarkan benua

,continent,average_life_expectancy
0,North America,72.99189
1,Asia,67.44118
2,Africa,52.57193
3,Europe,75.14773
4,South America,70.94615
5,Oceania,69.71500
6,Antarctica,NaN


In [7]:
sql(''' 
SELECT Continent, AVG(LifeExpectancy) as Average_Life_Expectancy FROM Country
GROUP BY Continent;
''')

,Continent,Average_Life_Expectancy
0,North America,72.99189
1,Asia,67.44118
2,Africa,52.57193
3,Europe,75.14773
4,South America,70.94615
5,Oceania,69.71500
6,Antarctica,NaN


## Tampilkan Jumlah region di masing-masing benua

,continent,jumlah_region
0,Asia,4
1,Europe,6
2,North America,3
3,Africa,5
4,Oceania,5
5,Antarctica,1
6,South America,1


In [9]:
sql(''' 
SELECT Continent, COUNT(DISTINCT(Region)) as Jumlah_Region FROM Country
GROUP BY Continent;
''')

,Continent,Jumlah_Region
0,Asia,4
1,Europe,6
2,North America,3
3,Africa,5
4,Oceania,5
5,Antarctica,1
6,South America,1


## Tampilkan rata-rata GNP di Afrika berdasarkan regionnya

,region,average_GNP
0,Central Africa,3659.78
1,Eastern Africa,3496.25
2,Western Africa,6277.12
3,Southern Africa,25386.20
4,Northern Africa,34838.57


In [15]:
sql(''' 
SELECT Region, AVG(GNP) as Average_GNP FROM Country WHERE Continent="Africa"
GROUP BY Region;
''')

,Region,Average_GNP
0,Central Africa,3659.777778
1,Eastern Africa,3496.250000
2,Western Africa,6277.117647
3,Southern Africa,25386.200000
4,Northern Africa,34838.571429


## Tampilkan negara di Eropa yang memiliki nama dimulai dari huruf I

,Negara
0,Ireland
1,Iceland
2,Italy


In [16]:
sql(''' 
SELECT Name as Negara FROM Country WHERE Continent="Europe" and Name LIKE "I%"
''')

,Negara
0,Ireland
1,Iceland
2,Italy


## Tampilkan 10 negara dengan GNP di atas rata-rata GNP negara-negara di benua Oceania, diurutkan dari yang terbesar

,negara,GNP
0,United States,8510700.0
1,Japan,3787042.0
2,Germany,2133367.0
3,France,1424285.0
4,United Kingdom,1378330.0
5,Italy,1161755.0
6,China,982268.0
7,Brazil,776739.0
8,Canada,598862.0
9,Spain,553233.0


In [19]:
sql(''' 
SELECT Name as Negara, GNP FROM Country
GROUP By Name HAVING GNP>(SELECT AVG(GNP) FROM Country WHERE Continent="Oceania")
ORDER BY GNP DESC LIMIT 10;
''')

,Negara,GNP
0,United States,8510700.0
1,Japan,3787042.0
2,Germany,2133367.0
3,France,1424285.0
4,United Kingdom,1378330.0
5,Italy,1161755.0
6,China,982268.0
7,Brazil,776739.0
8,Canada,598862.0
9,Spain,553233.0


## Tampikan GNP negara belanda, ibukota beserta populasi dari ibukotanya

,Country,GNP,Capital,population
0,Netherlands,371362.0,Amsterdam,731200


In [20]:
sql('''
SELECT Country.Name as Country, Country.GNP as GNP, City.Name as Capital, City.Population as Population
FROM Country, City
WHERE Country.Capital=City.ID and Country.Name="Netherlands";
''')

,Country,GNP,Capital,Population
0,Netherlands,371362.0,Amsterdam,731200


## 10 negara mana yang memiliki persentase pemakai bahasa spanyol paling tinggi

,negara,persentase
0,Cuba,100.0
1,El Salvador,100.0
2,Colombia,99.0
3,Dominican Republic,98.0
4,Nicaragua,97.6
5,Costa Rica,97.5
6,Honduras,97.2
7,Venezuela,96.9
8,Argentina,96.8
9,Uruguay,95.7


In [23]:
sql('''
SELECT Country.Name as Negara, CountryLanguage.Percentage as Persentase FROM Country, CountryLanguage
WHERE Country.Code=CountryLanguage.CountryCode and Language="Spanish" 
ORDER BY Persentase DESC LIMIT 10;
''')

,Negara,Persentase
0,Cuba,100.0
1,El Salvador,100.0
2,Colombia,99.0
3,Dominican Republic,98.0
4,Nicaragua,97.6
5,Costa Rica,97.5
6,Honduras,97.2
7,Venezuela,96.9
8,Argentina,96.8
9,Uruguay,95.7


## Tampikan GNP negara Indonesia, ibukota, populasi dari ibukotanya dan bahasa resmi yang dipakai

,negara,ibukota,populasi,bahasa
0,Indonesia,Jakarta,9604900,Malay


In [25]:
sql('''
SELECT Country.Name as Negara, City.Name as Ibukota, City.Population as Populasi, CountryLanguage.Language as Bahasa
FROM Country, City, CountryLanguage
WHERE Country.Capital=City.ID and Country.Code=CountryLanguage.CountryCode and
CountryLanguage.IsOfficial="T" and Country.Name = "Indonesia";
''')

,Negara,Ibukota,Populasi,Bahasa
0,Indonesia,Jakarta,9604900,Malay
